In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pandas as pd

per_eeg = []
per_emg = []
per_eog = []

file = input("The path to the .pkl file")
file = pickle.load(file)

ori_eeg = file[0]['ori_train_eegs']
syn_eeg = file[3]['syn_train_eegs']

ori_emg = file[1]['ori_train_emgs']
syn_emg = file[4]['syn_train_emgs']

ori_eog = file[2]['ori_train_eogs']
syn_eog = file[5]['syn_train_eogs']

for j, key in enumerate(ori_eeg.keys()):
    temp_subj_ori = key
    temp_subj_syn = [x for x in syn_eeg if key in x]

    print(temp_subj_ori)
    print(temp_subj_syn)
    print()

    ori_cor_mat_eeg = np.corrcoef(ori_eeg[temp_subj_ori], rowvar=False)
    ori_cor_mat_emg = np.corrcoef(ori_emg[temp_subj_ori], rowvar=False)
    ori_cor_mat_eog = np.corrcoef(ori_eog[temp_subj_ori], rowvar=False)

    data_eeg = []
    data_emg = []
    data_eog = []
    
    outliers_percent_up_eeg = []
    outliers_percent_up_emg = []
    outliers_percent_up_eog = []

    for s in temp_subj_syn:
        syn_cor_mat_eeg = np.corrcoef(syn_eeg[s], rowvar=False)
        syn_cor_mat_emg = np.corrcoef(syn_emg[s], rowvar=False)
        syn_cor_mat_eog = np.corrcoef(syn_eog[s], rowvar=False)
        
        corr_diff_eeg = np.abs(ori_cor_mat_eeg - syn_cor_mat_eeg)
        mean_corr_eeg = np.mean(corr_diff_eeg, axis=1)
        corr_diff_emg = np.abs(ori_cor_mat_emg - syn_cor_mat_emg)
        mean_corr_emg = np.mean(corr_diff_emg, axis=1)
        corr_diff_eog = np.abs(ori_cor_mat_eog - syn_cor_mat_eog)
        mean_corr_eog = np.mean(corr_diff_eog, axis=1)

        flat_corr_eeg = corr_diff_eeg.flatten()
        outliers_up_eeg = np.sum(np.where(flat_corr_eeg>0.3,1,0))
        out_per_up_eeg = (outliers_up_eeg)/len(flat_corr_eeg)*100
        flat_corr_emg = corr_diff_emg.flatten()
        outliers_up_emg = np.sum(np.where(flat_corr_emg>0.3,1,0))
        out_per_up_emg = (outliers_up_emg)/len(flat_corr_emg)*100
        flat_corr_eog = corr_diff_eog.flatten()
        outliers_up_eog = np.sum(np.where(flat_corr_eog>0.3,1,0))
        out_per_up_eog = (outliers_up_eog)/len(flat_corr_eog)*100
        
        data_eeg.append(mean_corr_eeg)
        outliers_percent_up_eeg.append(np.round(out_per_up_eeg,2))
        data_emg.append(mean_corr_emg)
        outliers_percent_up_emg.append(np.round(out_per_up_emg,2))
        data_eog.append(mean_corr_eog)
        outliers_percent_up_eog.append(np.round(out_per_up_eog,2))

        print(f'Subject {s} is complete')

    print()

    plt.figure(figsize=(10, 8))
    sns.kdeplot(data_eeg, palette='icefire', alpha=0.5, fill =True)
    # sns.barplot(data=np.array(data), color=crest_palette[j], alpha=0.6, fill=True)
    l = plt.legend([v[-4:] for v in temp_subj_syn])
    l.set_title(key)
    plt.title(f"Difference in Pairwise Correlations Between Original and Synthetic Data generated from {key}, EEG")
    plt.savefig(f"Difference in Pairwise Correlations Between Original and Synthetic Data generated from {key}, EEG.svg")
    plt.savefig(f"Difference in Pairwise Correlations Between Original and Synthetic Data generated from {key}, EEG.png")

    plt.figure(figsize=(10, 8))
    sns.kdeplot(data_emg, palette='rocket_r', alpha=0.5, fill =True)
    # sns.barplot(data=np.array(data), color=crest_palette[j], alpha=0.6, fill=True)
    l = plt.legend([v[-4:] for v in temp_subj_syn])
    l.set_title(key)
    plt.title(f"Difference in Pairwise Correlations Between Original and Synthetic Data generated from {key}, EMG")
    plt.savefig(f"Difference in Pairwise Correlations Between Original and Synthetic Data generated from {key}, EMG.svg")
    plt.savefig(f"Difference in Pairwise Correlations Between Original and Synthetic Data generated from {key}, EMG.png")

    plt.figure(figsize=(10, 8))
    sns.kdeplot(data_eog, palette='viridis', alpha=0.5, fill =True)
    # sns.barplot(data=np.array(data), color=crest_palette[j], alpha=0.6, fill=True)
    l = plt.legend([v[-4:] for v in temp_subj_syn])
    l.set_title(key)
    plt.title(f"Difference in Pairwise Correlations Between Original and Synthetic Data generated from {key}, EOG")
    plt.savefig(f"Difference in Pairwise Correlations Between Original and Synthetic Data generated from {key}, EOG.svg")
    plt.savefig(f"Difference in Pairwise Correlations Between Original and Synthetic Data generated from {key}, EOG.png")

    per_eeg.append(outliers_percent_up_eeg)
    per_emg.append(outliers_percent_up_emg)
    per_eog.append(outliers_percent_up_eog)


    df = pd.DataFrame({'EEG':outliers_percent_up_eeg, 'EMG':outliers_percent_up_emg, 'EOG': outliers_percent_up_eog})
    df.plot.bar(figsize=(10, 8), cmap='BuGn', alpha=0.8, edgecolor='black')
    plt.ylabel('Percentage, %')
    # plt.xlabel(f'Synthetic subjects generated from {key}')
    l = plt.legend()
    l.set_title(f'{key}')
    # plt.show()
    plt.xticks(np.arange(len(temp_subj_syn)), [v[-4:] for v in temp_subj_syn])
    plt.savefig(f"Percentage of values above 0.3, corr_mat {key}.svg")
    plt.savefig(f"Percentage of values above 0.3, corr_mat {key}.png")
       
       